In [1]:
# Import dependencies for web scraping and data cleaning

import pandas as pd
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
url = 'https://www.bestbuy.com/site/computer-cards-components/computer-pc-processors/abcat0507010.c?id=abcat0507010'
browser.visit(url)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Driver [C:\Users\rache\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


In [3]:
# Build a loop to go through each page (splinter) and append cpu names as well as prices into two seperate lists
cpu_list = []
price_list = []
html = browser.html
soup = BeautifulSoup(html,'html.parser')
cpus = soup.find_all("div", class_= 'sku-attribute-title')
prices = soup.find_all('div',class_='priceView-hero-price priceView-customer-price')
for cpu in cpus:
    cpu_list.append(cpu.text)    
for price in prices:
    price_list.append(price.text)   

In [4]:
print(cpu_list)
print(price_list)
print(len(cpu_list))
print(len(price_list))

['Model: BX8071512900K', 'SKU: 6483663', 'Model: 100-100000059WOF', 'SKU: 6438941', 'Model: 100-100000063WOF', 'SKU: 6439000', 'Model: BX8071512700K', 'SKU: 6483672', 'Model: BX8070110900K', 'SKU: 6411492', 'Model: BX8071512600K', 'SKU: 6483676', 'Model: BX8070811700K', 'SKU: 6452221', 'Model: BX8070110400', 'SKU: 6411498', 'Model: BX8070110900', 'SKU: 6411491', 'Model: 100-100000065BOX', 'SKU: 6438943', 'Model: BX8070811600K', 'SKU: 6452220', 'Model: BX8070110100', 'SKU: 6411497', 'Model: BX8070110700K', 'SKU: 6411488', 'Model: BX8070811900K', 'SKU: 6452222', 'Model: 100-100000061WOF', 'SKU: 6438942', 'Model: BX8070811900', 'SKU: 6452217', 'Model: BX8070811700', 'SKU: 6452219', 'Model: BX8071512900KF', 'SKU: 6483666', 'Model: BX8070811400', 'SKU: 6452218', 'Model: 100-100000263BOX', 'SKU: 6474406', 'Model: BX8071512400F', 'SKU: 6492588']
['$619.99Your price for this item is $619.99', '$714.99Your price for this item is $714.99', '$359.99Your price for this item is $359.99', '$419.99Yo

In [5]:
new_cpu_list =cpu_list[::2]
remove_indices = [18,20,23]
new_price_list = [i for j, i in enumerate(price_list) if j not in remove_indices]

In [6]:
url2 = 'https://www.bestbuy.com/site/searchpage.jsp?_dyncharset=UTF-8&browsedCategory=abcat0507010&cp=2&id=pcat17071&iht=n&ks=960&list=y&sc=Global&st=categoryid%24abcat0507010&type=page&usc=All%20Categories'
browser.visit(url2)
html = browser.html
soup = BeautifulSoup(html,'html.parser')

In [7]:
cpu_list2=[]
price_list2=[]
cpus2 = soup.find_all("div", class_= 'sku-attribute-title')
prices2 = soup.find_all('div',class_='priceView-hero-price priceView-customer-price')
for cpu in cpus2:
    cpu_list2.append(cpu.text)    
for price in prices2:
    price_list2.append(price.text)   

In [8]:
new_cpu_list2 =cpu_list2[::2]
remove_indices2 = [4,5,9,20,21]
new_price_list2 = [i for j, i in enumerate(price_list2) if j not in remove_indices2]

In [9]:
combined_cpu_list = new_cpu_list+new_cpu_list2
combined_cpu_list
print(len(combined_cpu_list))

41


In [10]:
combined_price_list = new_price_list + new_price_list2
combined_price_list
print(len(combined_price_list))

41


In [11]:
df = pd.DataFrame()
df['CPU']=combined_cpu_list
df['Price'] = combined_price_list
df.head()

,CPU,Price
0,Model: BX8071512900K,$619.99Your price for this item is $619.99
1,Model: 100-100000059WOF,$714.99Your price for this item is $714.99
2,Model: 100-100000063WOF,$359.99Your price for this item is $359.99
3,Model: BX8071512700K,$419.99Your price for this item is $419.99
4,Model: BX8070110900K,$467.99Your price for this item is $467.99


In [12]:
df['CPU'] =df['CPU'].str.rsplit(': ', 1).str[1].str[::1]
df['Price']=df['Price'].str.split('Y',1).str[0]
df['Price']=df['Price'].str.split('$',1).str[1]
df.head()

,CPU,Price
0,BX8071512900K,619.99
1,100-100000059WOF,714.99
2,100-100000063WOF,359.99
3,BX8071512700K,419.99
4,BX8070110900K,467.99


In [13]:
df.to_csv('best_buy_cpu.csv', index = False)